<a href="https://colab.research.google.com/github/UniVR-DH/LLM-test/blob/main/google_api_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
### Installation on Colab (As Of Oct)
!pip install -q -U langchain transformers bitsandbytes accelerate
!pip install -q -U langchain-community langchain-huggingface langchain-google-genai langgraph


In [7]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

In realms of code, a tale I'll sing,
Of LangChain, a language that's king.
A tapestry of words, it weaves,
A symphony of thought, it conceives.

In structure's embrace, it finds its might,
A hierarchy of blocks, so bright.
From basic elements to complex designs,
LangChain's architecture truly shines.

Its syntax, like a poet's quill,
Flows with grace, devoid of frill.
Indentation guides the reader's eye,
A visual cue that cannot lie.

But LangChain's strength lies not alone
In its structure or syntax, but its tone.
It whispers in the ears of AI,
A language they can comprehend and apply.

With each line of code, a task defined,
LangChain empowers minds, both vast and refined.
From data analysis to image recognition,
Its versatility defies any restriction.

In the hands of masters, it becomes a spell,
Transforming ideas into a digital swell.
Machine learning models take flight,
Guided by LangChain's illuminating light.

But like any language, it has its own quirks,
Its nuances and its su